# Distributed XGBoost (CPU)

woohoo!

In [ ]:
!pip install --upgrade dask distributed adlfs xgboost

## Get Workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Start Local

to delete

In [ ]:
from distributed import Client

c = Client()
c.restart()
c

In [ ]:
from adlfs import AzureBlobFileSystem

container_name = "malware"
storage_options = {"account_name": "azuremlexamples"}

fs = AzureBlobFileSystem(**storage_options)
fs

In [ ]:
files = fs.ls(f"{container_name}/process")
files

In [ ]:
import pandas as pd
import dask.dataframe as dd

In [ ]:
for f in files:
    if "train" in f:
        df_train = dd.read_parquet(f"az://{f}", storage_options=storage_options)
    elif "test" in f:
        df_test = dd.read_parquet(f"az://{f}", storage_options=storage_options)

df_train

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Get Data

We'll use ...